In [ ]:
import datetime as dt
from concurrent.futures import ProcessPoolExecutor
import numpy as np
import pandas as pd
import xlwings as xw
import pymysql
import calendar
import json
import math
import matplotlib.pyplot as plt
from scipy import optimize
import sys
import mplfinance as mpf



pd.options.display.float_format = '{:.4f}'.format

In [ ]:
# MySQL 서버에 연결
connection = pymysql.connect(
    host="127.0.0.1",  # MySQL 서버의 주소
    user="root",        # MySQL 사용자 이름
    port=3306,
    database='crypto',
    password="cdfha!3579",  # 비밀번호
    charset='utf8'
)
cur = connection.cursor()

query = "SELECT * FROM btcusdt1mf"
df = pd.read_sql(query, connection)

# 데이터 출력 확인
print(df)

# 연결 종료
connection.close()

df['Open time'] = pd.to_datetime(df['Open time'], unit ='ms')
df.set_index('Open time', inplace=True)
df = df[df['Volume'] > 0]

In [ ]:
def create_daily_timestamp_list(start_date, end_date):
    # 시작 날짜와 끝 날짜 사이의 일별 타임스탬프 리스트 생성
    daily_list = pd.date_range(start=start_date, end=end_date, freq='D')
    return daily_list

def red_black_3_candle(row):

    # 현재 값이 이전 값보다 크면 1, 그렇지 않으면 0
    first_candle_open = df['Open'].shift(2).loc[row.name]
    first_candle_close = df['Close'].shift(2).loc[row.name]

    second_candle_open = df['Open'].shift(1).loc[row.name]
    second_candle_close = df['Close'].shift(1).loc[row.name]

    third_candle_open = df['Open'].loc[row.name]
    third_candle_close = df['Close'].loc[row.name]
    
    if ((first_candle_close - first_candle_open) > 0) & ((second_candle_close - second_candle_open) > 0) & ((third_candle_close - third_candle_open) > 0) :
        if (third_candle_close > second_candle_close) & (second_candle_close > first_candle_close) :
            return 'red'
    
    elif ((first_candle_open - first_candle_close) > 0) & ((second_candle_open - second_candle_close) > 0) & ((third_candle_open - third_candle_close) > 0) :
        if (first_candle_close > second_candle_close) & (second_candle_close > third_candle_close) :
            return 'black'
    else :
        return None
    '''
    if ((first_candle_close - first_candle_open) > 0) & ((second_candle_close - second_candle_open) > 0) & ((third_candle_close - third_candle_open) > 0) :
        return 'red'
    
    elif ((first_candle_open - first_candle_close) > 0) & ((second_candle_open - second_candle_close) > 0) & ((third_candle_open - third_candle_close) > 0) :
        return 'black'
    else :
        return None
    '''

In [ ]:
df['red_black_3_candle'] = df.apply(lambda row : red_black_3_candle(row), axis=1)
df['day'] = df.index.normalize()

In [ ]:
daily_list = create_daily_timestamp_list('2024-01-01', '2024-12-31')
df_temp = df[df['day'].isin(daily_list)]

df_temp['MA3volumes'] = df_temp['Volume'].rolling(3).mean()
df_temp['MA5volumes'] = df_temp['Volume'].rolling(5).mean()
df_temp['MA10volumes'] = df_temp['Volume'].rolling(10).mean()
df_temp['MA20volumes'] = df_temp['Volume'].rolling(20).mean()
df_temp['MA30volumes'] = df_temp['Volume'].rolling(30).mean()
df_temp['power 5volume'] = np.where(df_temp['MA3volumes'] > df_temp['MA5volumes'], 1, 0)
df_temp['power 10volume'] = np.where(df_temp['MA3volumes'] > df_temp['MA10volumes'], 1, 0)
df_temp['power 20volume'] = np.where(df_temp['MA3volumes'] > df_temp['MA20volumes'], 1, 0)
df_temp['power 30volume'] = np.where(df_temp['MA3volumes'] > df_temp['MA30volumes'], 1, 0)

In [ ]:
daily_list = create_daily_timestamp_list('2024-01-01', '2024-12-31')
pl = 5000
current_position = None
for day in daily_list:
    df_day = df_temp[df_temp['day'] == day]
    index_list = df_day.index
    position_list = []
    for index, row in df_day.iterrows():

        current_idx = index_list.get_loc(index)

        if (current_idx+1 <len(df_day)) & (current_idx >= 1):
            next_index = index_list[current_idx+1]
            prior_index = index_list[current_idx-1]
            next_row = df_day.loc[next_index]
            prior_row = df_day.loc[prior_index]

            if (row['red_black_3_candle'] == 'red') & (row['power 20volume'] == 1):
                if current_position == None:
                    entry_price = next_row['Open']
                    current_position = 'red'
                    pl -= 5000 * 0.0002
                    position_list.append('red')
                elif current_position == 'black':
                    current_position = 'red'
                    pl += 5000 * (entry_price - row['Open'])/entry_price - 5000 * 0.0004
                    entry_price = next_row['Open']
                    position_list.append('red')
                elif current_position == 'red':
                    position_list.append(None)


            elif (row['red_black_3_candle'] == 'black') & (row['power 20volume'] == 1):
                if current_position == None:
                    entry_price = next_row['Open']
                    current_position = 'black'
                    pl -= 5000 * 0.0002
                    position_list.append('black')
                elif current_position == 'red':
                    current_position = 'black'
                    pl += 5000 * (row['Open'] - entry_price)/entry_price - 5000 * 0.0004
                    entry_price = next_row['Open']
                    position_list.append('black')
                elif current_position == 'black':
                    position_list.append(None)

            else:
                position_list.append(None)
            '''
            if (current_position == 'red') & (row['Close'] < prior_row['Open']):
                pl += next_row['Open'] - entry_price
                current_position = None


            elif (current_position == 'black') & (row['Close'] > prior_row['Open']):
                pl += entry_price - next_row['Open']
                current_position = None
            '''
        else:
            if current_position == 'red':
            # 매수 포지션 청산
                pl += 5000 * (row['Open'] - entry_price)/entry_price - 5000 * 0.0004
                position_list.append(None)
            elif current_position == 'black':
            # 매도 포지션 청산
                pl += 5000 * (entry_price - row['Open'])/entry_price - 5000 * 0.0004
                position_list.append(None)
            elif current_position == None:
                position_list.append(None)

    print(day, pl)  

    mpf_fig, mpf_axes = mpf.plot(
        df_day,
        type='candle',
        style='charles',
        volume=False,
        returnfig=True,
        figsize=(25, 15)
    )

    # 캔들 차트 축
    ax = mpf_axes[0]

    # red와 black 데이터 추출
    df_day['position'] = position_list
    red_points = df_day[df_day['position'] == 'red']
    black_points = df_day[df_day['position'] == 'black']
    

    # x축 좌표 생성 (mplfinance는 0부터 시작하는 정수로 x축을 매핑)
    x_coords = range(len(df_day))

    # 빨간색 화살표 추가 (red)
    for x, y in zip(x_coords, df_day['High']):
        if df_day['position'].iloc[x] == 'red':
            ax.scatter(
                x,
                y + 2,  # 캔들 위에 표시 (High 값보다 약간 위)
                color='red',
                label='Red' if 'Red' not in ax.get_legend_handles_labels()[1] else None,
                marker='v',  # 아래쪽 화살표
                zorder=5
            )

    # 파란색 화살표 추가 (black)
    for x, y in zip(x_coords, df_day['High']):
        if df_day['position'].iloc[x] == 'black':
            ax.scatter(
                x,
                y + 2,  # 캔들 위에 표시 (High 값보다 약간 위)
                color='blue',
                label='Black' if 'Black' not in ax.get_legend_handles_labels()[1] else None,
                marker='v',  # 아래쪽 화살표
                zorder=5
            )

    # 범례 추가
    ax.legend()

    # 차트 표시
    plt.show()
